In [3]:
!pip install pyro-ppl torch matplotlib numpy

In [6]:
# infinite_bayesian_operator.py

import torch
import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
import pyro.infer
import matplotlib.pyplot as plt
import numpy as np
import os

pyro.set_rng_seed(42)

# --- Step 1: Generate synthetic data (representing asset returns) ---

N = 50
X = torch.linspace(0, 1, N)
true_function = torch.sin(6 * X)  # True underlying function w0
Y = true_function + 0.1 * torch.randn(X.size())

# Save true function for diagnostic comparison
if not os.path.exists("diagnostics"):
    os.makedirs("diagnostics")
np.savetxt("diagnostics/true_function.txt", true_function.numpy())

# --- Step 2: Define GP prior over function-space weights ---

kernel = gp.kernels.RBF(input_dim=1, lengthscale=torch.tensor(0.2))
gpr = gp.models.GPRegression(X.unsqueeze(-1), Y, kernel, noise=torch.tensor(0.1))

# --- Step 3: Variational inference approximation to posterior using SVI ---

optimizer = pyro.optim.Adam({"lr": 0.01})
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(model=gpr.model, guide=gpr.guide, optim=optimizer, loss=elbo)

num_steps = 1500
loss_trace = []

print("Starting variational inference...")
for i in range(num_steps):
    loss = svi.step()
    loss_trace.append(loss)
    if i % 100 == 0:
        print(f"Step {i}: ELBO loss = {loss:.4f}")

# Save ELBO loss trace
np.savetxt("diagnostics/loss_trace.txt", np.array(loss_trace))

# --- Step 4: Predictive posterior diagnostics ---

X_test = torch.linspace(0, 1, 200).unsqueeze(-1)
mean, cov = gpr(X_test, full_cov=True)
std = cov.diag().sqrt()

# Coverage diagnostic: Check how often true function lies within 95% credible interval
lower = mean - 1.96 * std
upper = mean + 1.96 * std

# Interpolation of true function at test points
true_func_test = torch.sin(6 * X_test.squeeze())
coverage_hits = ((true_func_test >= lower) & (true_func_test <= upper)).float().mean()

print(f"Approximate coverage (95% band): {coverage_hits.item()*100:.2f}%")

# Save coverage diagnostic
with open("diagnostics/coverage.txt", "w") as f:
    f.write(f"Approximate coverage: {coverage_hits.item()*100:.2f}%\n")

# --- Step 5: Plot posterior mean and uncertainty bands ---

plt.figure(figsize=(10, 6))
plt.plot(X.numpy(), Y.numpy(), 'kx', label="Observations")
plt.plot(X_test.numpy(), mean.detach().numpy(), 'b', label="Posterior mean")
plt.fill_between(
    X_test.squeeze().numpy(),
    lower.detach().numpy(),
    upper.detach().numpy(),
    color='blue',
    alpha=0.2,
    label="95% credible band"
)
plt.plot(X_test.squeeze().numpy(), true_func_test.numpy(), 'r--', label="True function")
plt.title("Function-space posterior with uncertainty bands")
plt.legend()
plt.savefig("diagnostics/posterior_plot.png")
plt.close()

# --- Step 6: Save posterior mean and std to files for further analysis ---

np.savetxt("diagnostics/posterior_mean.txt", mean.detach().numpy())
np.savetxt("diagnostics/posterior_std.txt", std.detach().numpy())

# --- Step 7: Optional KL divergence approximation (variational diagnostic) ---

final_kl = loss_trace[-1]
with open("diagnostics/final_kl.txt", "w") as f:
    f.write(f"Approximate final negative ELBO (proxy KL): {final_kl:.4f}\n")

print("✅ Diagnostics and plots saved to 'diagnostics' folder.")
print("✅ Script complete. You can include these results in the proof appendix.")

Starting variational inference...
Step 0: ELBO loss = 3.2720
Step 100: ELBO loss = -16.4998
Step 200: ELBO loss = -27.6725
Step 300: ELBO loss = -30.7628
Step 400: ELBO loss = -30.9810
Step 500: ELBO loss = -30.9847
Step 600: ELBO loss = -30.9846
Step 700: ELBO loss = -30.9844
Step 800: ELBO loss = -30.9847
Step 900: ELBO loss = -30.9844
Step 1000: ELBO loss = -30.9848
Step 1100: ELBO loss = -30.9848
Step 1200: ELBO loss = -30.9848
Step 1300: ELBO loss = -30.9848
Step 1400: ELBO loss = -30.9847
Approximate coverage (95% band): 70.50%
✅ Diagnostics and plots saved to 'diagnostics' folder.
✅ Script complete. You can include these results in the proof appendix.


In [19]:
# stress_bayesian_operator.py

import torch
import pyro
import pyro.contrib.gp as gp
import pyro.infer
import matplotlib.pyplot as plt
import numpy as np
import os

pyro.set_rng_seed(42)

if not os.path.exists("stress_diagnostics"):
    os.makedirs("stress_diagnostics")

def diagnostics(X_test, mean, std, true_func):
    lower = mean - 1.96 * std
    upper = mean + 1.96 * std
    coverage = ((true_func >= lower) & (true_func <= upper)).float().mean().item() * 100
    mse = torch.mean((mean - true_func) ** 2).item()
    avg_std = torch.mean(std).item()
    return coverage, mse, avg_std

sample_sizes = [50, 200, 500, 1000, 2000, 3000]
noise_type = 'student_t'
true_function_type = 'sin'

for N in sample_sizes:
    print(f"\n🔬 Running stress test with N = {N} ...")

    pyro.clear_param_store()

    X = torch.linspace(0, 1, N)

    if true_function_type == 'sin':
        true_function = torch.sin(6 * X)
    elif true_function_type == 'piecewise':
        true_function = torch.where(X < 0.5, 0.5 * torch.ones_like(X), -0.5 * torch.ones_like(X))
    elif true_function_type == 'jump':
        true_function = torch.sin(6 * X)
        true_function[X > 0.7] += 1.0
    else:
        raise ValueError("Unsupported true function type.")

    if noise_type == 'gaussian':
        Y = true_function + 0.1 * torch.randn(X.size())
    elif noise_type == 'student_t':
        Y = true_function + 0.1 * torch.distributions.StudentT(df=3).sample(X.size())
    else:
        raise ValueError("Unsupported noise type.")

    kernel = gp.kernels.RBF(input_dim=1, lengthscale=torch.tensor(0.2))
    kernel.jitter = 1e-2  # High jitter

    # ✅ FINAL: Fixed, small number of inducing points with fixed positions
    if N <= 25:
        num_inducing = 5
    elif N <= 200:
        num_inducing = 10
    else:
        num_inducing = 15
    Xu = torch.linspace(0, 1, num_inducing).unsqueeze(-1)

    likelihood = gp.likelihoods.Gaussian()
    likelihood.noise = torch.tensor(0.1)

    gpr = gp.models.VariationalSparseGP(X.unsqueeze(-1), Y, kernel, Xu, likelihood)

    optimizer = pyro.optim.Adam({"lr": 0.01})
    elbo = pyro.infer.Trace_ELBO()
    svi = pyro.infer.SVI(model=gpr.model, guide=gpr.guide, optim=optimizer, loss=elbo)

    num_steps = 1500
    loss_trace = []

    for i in range(num_steps):
        loss = svi.step()
        loss_trace.append(loss)

    X_test = torch.linspace(0, 1, 500).unsqueeze(-1)
    mean, cov = gpr(X_test, full_cov=True)
    std = cov.diag().sqrt()

    true_func_test = torch.sin(6 * X_test.squeeze())
    if true_function_type == 'piecewise':
        true_func_test = torch.where(X_test.squeeze() < 0.5, 0.5, -0.5)
    elif true_function_type == 'jump':
        true_func_test = torch.sin(6 * X_test.squeeze())
        true_func_test[X_test.squeeze() > 0.7] += 1.0

    coverage, mse, avg_std = diagnostics(X_test.squeeze(), mean, std, true_func_test)

    print(f"N = {N} | Coverage: {coverage:.2f}% | MSE: {mse:.4f} | Avg Std: {avg_std:.4f}")

    np.savetxt(f"stress_diagnostics/loss_trace_N{N}.txt", np.array(loss_trace))
    np.savetxt(f"stress_diagnostics/posterior_mean_N{N}.txt", mean.detach().numpy())
    np.savetxt(f"stress_diagnostics/posterior_std_N{N}.txt", std.detach().numpy())

    with open(f"stress_diagnostics/summary_N{N}.txt", "w") as f:
        f.write(f"Coverage: {coverage:.2f}%\n")
        f.write(f"MSE: {mse:.4f}\n")
        f.write(f"Avg posterior std: {avg_std:.4f}\n")

    plt.figure(figsize=(10, 6))
    plt.plot(X.numpy(), Y.numpy(), 'kx', label="Observations")
    plt.plot(X_test.numpy(), mean.detach().numpy(), 'b', label="Posterior mean")
    plt.fill_between(
        X_test.squeeze().numpy(),
        (mean - 1.96 * std).detach().numpy(),
        (mean + 1.96 * std).detach().numpy(),
        color='blue',
        alpha=0.2,
        label="95% credible band"
    )
    plt.plot(X_test.squeeze().numpy(), true_func_test.numpy(), 'r--', label="True function")
    plt.title(f"Posterior (N={N})")
    plt.legend()
    plt.savefig(f"stress_diagnostics/posterior_plot_N{N}.png")
    plt.close()


🔬 Running stress test with N = 50 ...
N = 50 | Coverage: 100.00% | MSE: 0.0425 | Avg Std: 0.5403

🔬 Running stress test with N = 200 ...
N = 200 | Coverage: 100.00% | MSE: 0.0008 | Avg Std: 0.2652

🔬 Running stress test with N = 500 ...
N = 500 | Coverage: 100.00% | MSE: 0.5081 | Avg Std: 0.8856

🔬 Running stress test with N = 1000 ...
N = 1000 | Coverage: 100.00% | MSE: 0.4886 | Avg Std: 0.8790

🔬 Running stress test with N = 2000 ...
N = 2000 | Coverage: 100.00% | MSE: 0.4545 | Avg Std: 0.8434

🔬 Running stress test with N = 3000 ...
N = 3000 | Coverage: 100.00% | MSE: 0.4384 | Avg Std: 0.8387


In [20]:
# extended_bayesian_operator_tests.py

import torch
import pyro
import pyro.contrib.gp as gp
import pyro.infer
import numpy as np
import matplotlib.pyplot as plt
import time
import os

pyro.set_rng_seed(42)

if not os.path.exists("extended_tests"):
    os.makedirs("extended_tests")

# ========================================
# Diagnostics
# ========================================
def diagnostics(X_test, mean, std, true_func):
    lower = mean - 1.96 * std
    upper = mean + 1.96 * std
    coverage = ((true_func >= lower) & (true_func <= upper)).float().mean().item() * 100
    mse = torch.mean((mean - true_func) ** 2).item()
    avg_std = torch.mean(std).item()
    return coverage, mse, avg_std

# ========================================
# Setup kernel and GP
# ========================================
def build_gp(X, Y, num_inducing, kernel_type='RBF', jitter=1e-2):
    if kernel_type == 'RBF':
        kernel = gp.kernels.RBF(input_dim=1, lengthscale=torch.tensor(0.2))
    elif kernel_type == 'Matern32':
        kernel = gp.kernels.Matern32(input_dim=1, lengthscale=torch.tensor(0.2))
    else:
        raise ValueError("Unsupported kernel type.")
    kernel.jitter = jitter

    Xu = torch.linspace(0, 1, num_inducing).unsqueeze(-1)
    likelihood = gp.likelihoods.Gaussian()
    likelihood.noise = torch.tensor(0.1)
    gpr = gp.models.VariationalSparseGP(X.unsqueeze(-1), Y, kernel, Xu, likelihood)
    return gpr

# ========================================
# True function generator
# ========================================
def true_function_generator(X, type='sin', drift=0.0):
    if type == 'sin':
        return torch.sin(6 * X + drift)
    elif type == 'piecewise':
        return torch.where(X < 0.5, 0.5 * torch.ones_like(X), -0.5 * torch.ones_like(X))
    elif type == 'jump':
        f = torch.sin(6 * X)
        f[X > 0.7] += 1.0
        return f
    else:
        raise ValueError("Unsupported true function type.")

# ========================================
# Test configurations
# ========================================
sample_sizes = [50, 200, 500, 1000]
noise_types = ['gaussian', 'student_t', 'laplace']
kernel_types = ['RBF', 'Matern32']

# ========================================
# Run all tests
# ========================================
for N in sample_sizes:
    for noise_type in noise_types:
        for kernel_type in kernel_types:
            print(f"\n🔥 Running extended test: N={N}, noise={noise_type}, kernel={kernel_type}")

            pyro.clear_param_store()

            X = torch.linspace(0, 1, N)
            true_f = true_function_generator(X, type='sin', drift=0.0)

            if noise_type == 'gaussian':
                Y = true_f + 0.1 * torch.randn(X.size())
            elif noise_type == 'student_t':
                Y = true_f + 0.1 * torch.distributions.StudentT(df=3).sample(X.size())
            elif noise_type == 'laplace':
                Y = true_f + 0.1 * torch.distributions.Laplace(0, 1).sample(X.size())
            else:
                raise ValueError("Unsupported noise type.")

            # Inducing points
            if N <= 100:
                num_inducing = 5
            else:
                num_inducing = 10

            # Build GP
            gpr = build_gp(X, Y, num_inducing, kernel_type=kernel_type, jitter=1e-2)

            optimizer = pyro.optim.Adam({"lr": 0.01})
            elbo = pyro.infer.Trace_ELBO()
            svi = pyro.infer.SVI(model=gpr.model, guide=gpr.guide, optim=optimizer, loss=elbo)

            num_steps = 500
            loss_trace = []
            t0 = time.time()

            for i in range(num_steps):
                loss = svi.step()
                loss_trace.append(loss)

            elapsed = time.time() - t0

            # Evaluate
            X_test = torch.linspace(0, 1, 500).unsqueeze(-1)
            mean, cov = gpr(X_test, full_cov=True)
            std = cov.diag().sqrt()
            true_f_test = true_function_generator(X_test.squeeze(), type='sin', drift=0.0)

            coverage, mse, avg_std = diagnostics(X_test.squeeze(), mean, std, true_f_test)

            print(f"✅ Coverage: {coverage:.2f}% | MSE: {mse:.4f} | Avg Std: {avg_std:.4f} | Time: {elapsed:.1f}s")

            # Save
            label = f"N{N}_noise{noise_type}_kernel{kernel_type}"
            np.savetxt(f"extended_tests/loss_trace_{label}.txt", np.array(loss_trace))
            np.savetxt(f"extended_tests/posterior_mean_{label}.txt", mean.detach().numpy())
            np.savetxt(f"extended_tests/posterior_std_{label}.txt", std.detach().numpy())

            plt.figure(figsize=(10, 6))
            plt.plot(X.numpy(), Y.numpy(), 'kx', label="Observations")
            plt.plot(X_test.numpy(), mean.detach().numpy(), 'b', label="Posterior mean")
            plt.fill_between(
                X_test.squeeze().numpy(),
                (mean - 1.96 * std).detach().numpy(),
                (mean + 1.96 * std).detach().numpy(),
                color='blue',
                alpha=0.2,
                label="95% credible band"
            )
            plt.plot(X_test.squeeze().numpy(), true_f_test.numpy(), 'r--', label="True function")
            plt.title(f"N={N}, noise={noise_type}, kernel={kernel_type}")
            plt.legend()
            plt.savefig(f"extended_tests/posterior_plot_{label}.png")
            plt.close()


🔥 Running extended test: N=50, noise=gaussian, kernel=RBF
✅ Coverage: 100.00% | MSE: 0.0023 | Avg Std: 0.0700 | Time: 5.8s

🔥 Running extended test: N=50, noise=gaussian, kernel=Matern32
✅ Coverage: 100.00% | MSE: 0.0036 | Avg Std: 0.0993 | Time: 5.5s

🔥 Running extended test: N=50, noise=student_t, kernel=RBF
✅ Coverage: 100.00% | MSE: 0.0029 | Avg Std: 0.1649 | Time: 5.2s

🔥 Running extended test: N=50, noise=student_t, kernel=Matern32
✅ Coverage: 100.00% | MSE: 0.0035 | Avg Std: 0.1541 | Time: 5.1s

🔥 Running extended test: N=50, noise=laplace, kernel=RBF
✅ Coverage: 100.00% | MSE: 0.0023 | Avg Std: 0.1026 | Time: 5.7s

🔥 Running extended test: N=50, noise=laplace, kernel=Matern32
✅ Coverage: 100.00% | MSE: 0.0019 | Avg Std: 0.1144 | Time: 5.1s

🔥 Running extended test: N=200, noise=gaussian, kernel=RBF
✅ Coverage: 100.00% | MSE: 0.1625 | Avg Std: 0.6968 | Time: 5.9s

🔥 Running extended test: N=200, noise=gaussian, kernel=Matern32
✅ Coverage: 100.00% | MSE: 0.0012 | Avg Std: 0.0730

In [26]:
# ultimate_bayesian_operator_tests.py

import torch
import pyro
import pyro.contrib.gp as gp
import pyro.infer
import numpy as np
import matplotlib.pyplot as plt
import os
import time

pyro.set_rng_seed(42)

if not os.path.exists("ultimate_tests"):
    os.makedirs("ultimate_tests")

def diagnostics(X_test, mean, std, true_func):
    lower = mean - 1.96 * std
    upper = mean + 1.96 * std
    coverage = ((true_func >= lower) & (true_func <= upper)).float().mean().item() * 100
    mse = torch.mean((mean - true_func) ** 2).item()
    avg_std = torch.mean(std).item()
    return coverage, mse, avg_std

def build_gp(X, Y, Xu, kernel):
    likelihood = gp.likelihoods.Gaussian()
    likelihood.noise = torch.tensor(0.1)
    gpr = gp.models.VariationalSparseGP(X.unsqueeze(-1), Y, kernel, Xu, likelihood)
    return gpr

def true_function_generator(X, type='sin', drift=0.0):
    if type == 'sin':
        return torch.sin(6 * X + drift)
    elif type == 'piecewise':
        return torch.where(X < 0.5, 0.5 * torch.ones_like(X), -0.5 * torch.ones_like(X))
    elif type == 'jump':
        f = torch.sin(6 * X)
        f[X > 0.7] += 1.0
        return f
    else:
        raise ValueError("Unsupported true function type.")

def posterior_sample_trajectories(gpr, X_test, num_samples=10):
    mean, cov = gpr(X_test, full_cov=True)
    cov = cov + 1e-4 * torch.eye(cov.shape[0], device=cov.device)
    mvn = torch.distributions.MultivariateNormal(mean, covariance_matrix=cov)
    samples = []
    for _ in range(num_samples):
        f_samp = mvn.sample()
        samples.append(f_samp.squeeze().detach().cpu().numpy())
    return samples

def compute_eigenvalues(kernel, Xu):
    Kuu = kernel(Xu).contiguous().detach().cpu().numpy()
    eigvals = np.linalg.eigvalsh(Kuu + np.eye(Kuu.shape[0]) * kernel.jitter)
    return eigvals

sample_sizes = [50, 200, 500]
noise_types = ['gaussian', 'student_t', 'laplace']
kernel_types = ['RBF', 'Matern32']

# ====================================================
# Run all tests
# ====================================================
for N in sample_sizes:
    for noise_type in noise_types:
        for kernel_type in kernel_types:
            print(f"\n🔥 Ultimate test: N={N}, noise={noise_type}, kernel={kernel_type}")

            pyro.clear_param_store()

            X = torch.linspace(0, 1, N)
            true_f = true_function_generator(X, type='sin', drift=0.0)

            if noise_type == 'gaussian':
                Y = true_f + 0.1 * torch.randn(X.size())
            elif noise_type == 'student_t':
                Y = true_f + 0.1 * torch.distributions.StudentT(df=3).sample(X.size())
            elif noise_type == 'laplace':
                Y = true_f + 0.1 * torch.distributions.Laplace(0, 1).sample(X.size())
            else:
                raise ValueError("Unsupported noise type.")

            # ✅ Final ultimate fix for inducing points and kernel
            if N <= 100:
                num_inducing = 5
            else:
                num_inducing = 10
            Xu = torch.linspace(0, 1, num_inducing).unsqueeze(-1)

            if kernel_type == 'RBF':
                kernel = gp.kernels.RBF(input_dim=1, lengthscale=torch.tensor(0.2))
            elif kernel_type == 'Matern32':
                kernel = gp.kernels.Matern32(input_dim=1, lengthscale=torch.tensor(0.2))
            else:
                raise ValueError("Unsupported kernel type.")

            kernel.jitter = 1e-1  # 🔥 large jitter for numerical stability

            gpr = build_gp(X, Y, Xu, kernel)

            optimizer = pyro.optim.Adam({"lr": 0.01})
            elbo = pyro.infer.Trace_ELBO()
            svi = pyro.infer.SVI(model=gpr.model, guide=gpr.guide, optim=optimizer, loss=elbo)

            num_steps = 500
            loss_trace = []
            t0 = time.time()

            for i in range(num_steps):
                loss = svi.step()
                loss_trace.append(loss)

            elapsed = time.time() - t0

            X_test = torch.linspace(0, 1, 500).unsqueeze(-1)
            mean, cov = gpr(X_test, full_cov=True)
            std = cov.diag().sqrt()
            true_f_test = true_function_generator(X_test.squeeze(), type='sin', drift=0.0)

            coverage, mse, avg_std = diagnostics(X_test.squeeze(), mean, std, true_f_test)

            # Posterior sample trajectories
            samples = posterior_sample_trajectories(gpr, X_test, num_samples=10)

            # Eigenvalues
            eigvals = compute_eigenvalues(kernel, Xu)

            print(f"✅ Coverage: {coverage:.2f}% | MSE: {mse:.4f} | Avg Std: {avg_std:.4f} | Time: {elapsed:.1f}s")

            # Save everything
            label = f"N{N}_noise{noise_type}_kernel{kernel_type}"
            np.savetxt(f"ultimate_tests/loss_trace_{label}.txt", np.array(loss_trace))
            np.savetxt(f"ultimate_tests/posterior_mean_{label}.txt", mean.detach().numpy())
            np.savetxt(f"ultimate_tests/posterior_std_{label}.txt", std.detach().numpy())
            np.savetxt(f"ultimate_tests/eigenvalues_{label}.txt", eigvals)

            plt.figure(figsize=(10, 6))
            plt.plot(X.numpy(), Y.numpy(), 'kx', label="Observations")
            plt.plot(X_test.numpy(), mean.detach().numpy(), 'b', label="Posterior mean")
            plt.fill_between(
                X_test.squeeze().numpy(),
                (mean - 1.96 * std).detach().numpy(),
                (mean + 1.96 * std).detach().numpy(),
                color='blue',
                alpha=0.2,
                label="95% credible band"
            )
            plt.plot(X_test.squeeze().numpy(), true_f_test.numpy(), 'r--', label="True function")
            for samp in samples:
                plt.plot(X_test.squeeze().numpy(), samp[:len(X_test)], 'g', alpha=0.3)
            plt.title(f"N={N}, noise={noise_type}, kernel={kernel_type}")
            plt.legend()
            plt.savefig(f"ultimate_tests/posterior_plot_{label}.png")
            plt.close()

            plt.figure(figsize=(8, 5))
            plt.plot(np.sort(eigvals)[::-1], 'o-', label="Kernel eigenvalues")
            plt.yscale("log")
            plt.title(f"Eigenvalue decay - {label}")
            plt.xlabel("Index")
            plt.ylabel("Eigenvalue (log scale)")
            plt.legend()
            plt.savefig(f"ultimate_tests/eigen_plot_{label}.png")
            plt.close()


🔥 Ultimate test: N=50, noise=gaussian, kernel=RBF
✅ Coverage: 100.00% | MSE: 0.0023 | Avg Std: 0.0700 | Time: 6.6s

🔥 Ultimate test: N=50, noise=gaussian, kernel=Matern32
✅ Coverage: 100.00% | MSE: 0.0024 | Avg Std: 0.0925 | Time: 5.7s

🔥 Ultimate test: N=50, noise=student_t, kernel=RBF
✅ Coverage: 100.00% | MSE: 0.0009 | Avg Std: 0.1181 | Time: 8.3s

🔥 Ultimate test: N=50, noise=student_t, kernel=Matern32
✅ Coverage: 100.00% | MSE: 0.0032 | Avg Std: 0.1127 | Time: 5.4s

🔥 Ultimate test: N=50, noise=laplace, kernel=RBF
✅ Coverage: 100.00% | MSE: 0.0020 | Avg Std: 0.1021 | Time: 6.0s

🔥 Ultimate test: N=50, noise=laplace, kernel=Matern32
✅ Coverage: 100.00% | MSE: 0.0023 | Avg Std: 0.1251 | Time: 7.0s

🔥 Ultimate test: N=200, noise=gaussian, kernel=RBF
✅ Coverage: 100.00% | MSE: 0.0443 | Avg Std: 0.5501 | Time: 5.7s

🔥 Ultimate test: N=200, noise=gaussian, kernel=Matern32
✅ Coverage: 100.00% | MSE: 0.0005 | Avg Std: 0.0678 | Time: 6.6s

🔥 Ultimate test: N=200, noise=student_t, kernel=R